Name : Vani Balani

Roll No : 21BCE021

Course & Course Code : 2CSDE93

Practical No : 3

AIM : To create a blockchain and consensus mechanism using POW and reward distribution


In [ ]:
import hashlib
import datetime

In [ ]:
class Block:
    def __init__(self, index, previous_hash, timestamp, transaction_data, current_hash, nonce):
        self.index = index
        self.previous_hash = previous_hash
        self.timestamp = timestamp
        self.transaction_data = transaction_data
        self.current_hash = current_hash
        self.nonce = nonce

    def __repr__(self):
        return f"Block(Index: {self.index}, Hash: {self.current_hash}, Previous Hash: {self.previous_hash}, Timestamp: {self.timestamp}, Data: {self.transaction_data}, Nonce: {self.nonce})"

class Blockchain:
    def __init__(self, difficulty_level=2, reward=50):
        self.chain = [self.generate_genesis_block()]
        self.difficulty_level = difficulty_level
        self.reward = reward
        self.pending_rewards = {}

    def generate_genesis_block(self):
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        return Block(0, "0", timestamp, "Initial Block", self.compute_block_hash(0, "0", timestamp, "Initial Block", 0), 0)

    def retrieve_last_block(self):
        return self.chain[-1]

    def append_block(self, transaction_data):
        last_block = self.retrieve_last_block()
        new_index = last_block.index + 1
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        previous_hash = last_block.current_hash
        nonce, current_hash = self.compute_proof_of_work(new_index, previous_hash, timestamp, transaction_data)
        new_block = Block(new_index, previous_hash, timestamp, transaction_data, current_hash, nonce)
        self.chain.append(new_block)

    def add_block(self, data, miner_address):
        new_block = self.append_block(data)
        self.chain.append(new_block)
        self.pending_rewards[miner_address] = self.pending_rewards.get(miner_address, 0) + self.reward

    def compute_block_hash(self, index, previous_hash, timestamp, transaction_data, nonce):
        value = f"{index}{previous_hash}{timestamp}{transaction_data}{nonce}"
        return hashlib.sha256(value.encode('utf-8')).hexdigest()

    def compute_proof_of_work(self, index, previous_hash, timestamp, transaction_data):
        nonce = 0
        while True:
            current_hash = self.compute_block_hash(index, previous_hash, timestamp, transaction_data, nonce)
            if current_hash[:self.difficulty_level] == '0' * self.difficulty_level:
                return nonce, current_hash
            nonce += 1

    def validate_chain(self):
        for i in range(1, len(self.chain)):
            current_block = self.chain[i]
            previous_block = self.chain[i-1]
            recalculated_hash = self.compute_block_hash(current_block.index, current_block.previous_hash, current_block.timestamp, current_block.transaction_data, current_block.nonce)
            if current_block.current_hash != recalculated_hash:
                return False
            if current_block.previous_hash != previous_block.current_hash:
                return False
        return True

    def alter_block(self, index, updated_data):
        if 0 < index < len(self.chain):
            self.chain[index].transaction_data = updated_data
            self.update_block_hash(index)

    def update_block_hash(self, index):
        current_block = self.chain[index]
        previous_hash = self.chain[index-1].current_hash if index > 0 else "0"
        nonce, current_hash = self.compute_proof_of_work(current_block.index, previous_hash, current_block.timestamp, current_block.transaction_data)
        current_block.current_hash = current_hash
        current_block.nonce = nonce
        current_block.previous_hash = previous_hash

    def rewards(self, miner_address):
        return self.pending_rewards.get(miner_address, 0)

In [ ]:
def run_blockchain_operations():
    blockchain = Blockchain(difficulty_level=2)
    blockchain.append_block("Transaction 1: Alice pays Bob 10 BTC")
    blockchain.append_block("Transaction 2: Bob pays Charlie 5 BTC")
    blockchain.append_block("Transaction 3: Charlie pays Alice 2 BTC")

    print("Initial Blockchain:")
    for block in blockchain.chain:
        print(block)

    print("REWARDS")
    print(f"Miner1: {blockchain.rewards('Miner1')}")
    print(f"Miner2: {blockchain.rewards('Miner2')}")

    print("\nBlockchain validity -> ", blockchain.validate_chain())

    blockchain.alter_block(1, "Tampered Transaction: Alice pays Bob 100 BTC")
    print("\nBlockchain post-tampering:")
    for block in blockchain.chain:
        print(block)

    print("\nBlockchain validity -> ", blockchain.validate_chain())

run_blockchain_operations()

Initial Blockchain:
Block(Index: 0, Hash: 3b3a16457a3272322d665715513c5b276912c12677384607ccb2864506f86c2b, Previous Hash: 0, Timestamp: 2024-09-02 12:15:07, Data: Initial Block, Nonce: 0)
Block(Index: 1, Hash: 00e603f1e2f25dc0cc093b2a4453be45eef709e3b62529b607baaf1f66528fdb, Previous Hash: 3b3a16457a3272322d665715513c5b276912c12677384607ccb2864506f86c2b, Timestamp: 2024-09-02 12:15:07, Data: Transaction 1: Alice pays Bob 10 BTC, Nonce: 1252)
Block(Index: 2, Hash: 00b5a3094702c02ef5e173c4ecc23af247bb6c53b1925bc7bfd8156763a3a84b, Previous Hash: 00e603f1e2f25dc0cc093b2a4453be45eef709e3b62529b607baaf1f66528fdb, Timestamp: 2024-09-02 12:15:07, Data: Transaction 2: Bob pays Charlie 5 BTC, Nonce: 7)
Block(Index: 3, Hash: 00f71079dda8453bf70ea8a03913dc22e87038280f729186032d0d99aeb395ab, Previous Hash: 00b5a3094702c02ef5e173c4ecc23af247bb6c53b1925bc7bfd8156763a3a84b, Timestamp: 2024-09-02 12:15:07, Data: Transaction 3: Charlie pays Alice 2 BTC, Nonce: 51)
REWARDS
Miner1: 0
Miner2: 0

Blockchai